### Question Answer Application
The goal of Question Answering is to find the answer to a question given a question and an accompanying context. The predicted answer will be either a span of text from the context or an empty string (indicating the question cannot be answered from the context).

In [54]:
import json
with open(r"train-v1.1.json", "r") as read_file:
    train = json.load(read_file)

In [75]:
from datasets import load_dataset

datasets = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to C:/Users/Afnan Qasim/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [106]:
train = datasets["train"]
train



Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [111]:
data_list = []

for data_point in train:
    # Extract the relevant information from the data point
    id = data_point['id']
    title = data_point['title']
    context = data_point['context']
    question = data_point['question']
    answers = data_point['answers']

    # Create a list representation of the data point
    data_list.append([id, title, context, question, answers])




IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [113]:
# Print the converted list
print(data_list[0][1])

['University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notre_Dame', 'University_of_Notr

In [104]:
data = [
    {
        'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
        'qas': [
            {
                'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
                'answers': [
                    {
                        'text': 'Saint Bernadette Soubirous',
                        'answer_start': 515
                    }
                ]
            }
        ]
    }
]

# Convert to list format
train_data = []
for item in data:
    context = item['context']
    qas_list = item['qas']
    for qas in qas_list:
        question = qas['question']
        answer_list = qas['answers']
        for answer in answer_list:
            answer_text = answer['text']
            answer_start = answer['answer_start']
            train_data.append({
                'context': context,
                'question': question,
                'answers': {
                    'answer_start': answer_start,
                    'text': answer_text
                }
            })

print(train_data)


[{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}}]


In [93]:
with open(r"dev-v1.1.json", "r") as read_file:
    test = json.load(read_file)

In [94]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [95]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [96]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5
use_cuda = False


In [97]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
    "use_cuda": False
}

In [98]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args,use_cuda = False

)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [74]:
### Remove output folder
!rm -rf outputs

In [110]:
# Train the model
model.train_model(train, eval_data=data_list)

KeyError: "Column qas not in the dataset. Current columns in the dataset: ['id', 'title', 'context', 'question', 'answers']"

In [39]:
# Evaluate the model
result, texts = model.eval_model(test)

TypeError: Input should be a list of examples.

In [32]:
result

{'correct': 1, 'similar': 1, 'incorrect': 1, 'eval_loss': -1.209026575088501}

In [33]:
texts

{'correct_text': {'00003': ''},
 'similar_text': {'00001': {'truth': 'region called the Final Empire',
   'predicted': '',
   'question': 'Where does the series take place?'}},
 'incorrect_text': {'00002': {'truth': 'one',
   'predicted': '"Mistings"',
   'question': 'How many powers does a Misting possess?'}}}

In [38]:
# Make predictions with the model
to_predict = [
    {
        "context": "The series primarily takes place in a region called the Final Empire on a world called Scadrial, where the sun and sky are red, vegetation is brown, and the ground is constantly being covered under black volcanic ashfalls.",
        "qas": [
            {
                "question": "Where does the series take place?",
                "id": "0",
            }
        ],
    }
]

In [42]:
to_predict = []

context = input("Enter the context: ")

question = input("Enter the question: ")

qa = {
    "context": context,
    "qas": [
        {
            "question": question,
            "id": "0",
        }
    ],
}

to_predict.append(qa)


Enter the context: The sky is blue.
Enter the question: What color is the sky?


In [43]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<?, ?it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['empty']}]
